<a href="https://colab.research.google.com/github/omerhac/arc_challenge/blob/master/metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def pixelwise_error_loss(y_true, y_pred):
  """
  Custom loss function. Pixelwise error from the true image. ||y_true - y_pred||
  """

  return tf.keras.backend.sum((y_true - y_pred) ** 2) / y_pred.shape[0]


def pixelwise_accuracy(y_true, y_pred, threshold=0.8):
  """
  Custom accuracy function. Pixelwise accuracy of predicting 0 / 1
  """
  
  tp = tf.math.reduce_sum(tf.math.multiply(tf.cast((y_pred >= threshold), 'float32'), y_true)) # cast boolean to binary
  tn = tf.math.reduce_sum(tf.math.multiply(tf.cast((y_pred < threshold), 'float32'), tf.cast((y_true == 0), 'float32')))
  n_predictions = BOARD_SIZE[0] * BOARD_SIZE[1]

  return ((tp + tn) / n_predictions) / y_pred.shape[0]


def pixelwise_sensitivity(y_true, y_pred, threshold=0.8):
  """
  Pixelwise sensitivity of correctly predicting 1s. true positives / positives
  """

  tp = tf.math.reduce_sum(tf.math.multiply(tf.cast((y_pred >= threshold), 'float32'), y_true)) 
  p = tf.cast(tf.math.reduce_sum(y_true), 'float32') 
  
  return tf.cond(p > 0, lambda: tp / p, lambda: tf.constant(1, dtype='float32')) # don't divide by zero

def pixelwise_fpr(y_true, y_pred, threshold=0.8):
  """
  Pixelwise false positive rate of wrongly predicting 1s. false positives / negatives
  """

  fp = tf.math.reduce_sum(tf.math.multiply(tf.cast((y_pred >= threshold), 'float32'), tf.cast((y_true==0), 'float32'))) # cast boolean to binary
  n = tf.cast(tf.math.reduce_sum(tf.cast((y_true == 0), 'float32')), 'float32')

  return tf.cond(n > 0, lambda: fp / n, lambda: tf.constant(1, dtype='float32')) # don't divide by zero


def pixelwise_auc(y_true, y_pred, num_thresholds=10):
  """
  Pixelwise auc score. 
  """

  sensitivity = []
  fpr = []

  # compute graph points
  for threshold in np.linspace(1, 0, num=num_thresholds):
    sensitivity.append(pixelwise_sensitivity(y_true, y_pred, threshold=threshold))
    fpr.append(pixelwise_fpr(y_true, y_pred, threshold=threshold))
  
  # compute trapeze area
  trapeze = []
  for i in range(len(sensitivity)- 1):
    area = tf.multiply(sensitivity[i], fpr[i+1] - fpr[i])
    trapeze.append(area)
  
  auc_score = tf.add_n(trapeze) 

  return auc_score